In [59]:
statevec_dim = 8
random_statevec = np.random.randn(statevec_dim)
random_statevec /= np.linalg.norm(random_statevec)

n_qubits = np.log2(statevec_dim)

@qml.qnode(dev)
def math_circuit(state = random_statevec):
    qml.QubitStateVector(state, wires = list(range(n_qubits)))
    
    return qml.probs(wires = list(range(n_qubits)))

@qml.qnode(dev)
def implemented_circuit(state = random_statevec):
    qml.QubitStateVector.decomposition(state, wires = list(range(n_qubits)))
    
    return qml.probs(wires = list(range(n_qubits)))

In [40]:
import pennylane as qml
from pennylane import numpy as np

from torch.autograd.functional import jacobian

In [41]:
def cry_cascade(n_qubits, params):
    assert len(params) == n_qubits - 1, f"Need {n_qubits - 1} params"
    for i in range(n_qubits - 1):
        qml.CRY(params[i], wires = [i, n_qubits - 1])

In [42]:
n_qubits = 4

dev = qml.device('default.qubit', wires = n_qubits)

def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

@qml.qnode(dev)
def circuit(params , input_vec=None):
    
    params = np.random.randn(n_qubits - 1)
    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node
    RY_layer(input_vec)    

    cry_cascade(n_qubits = n_qubits, params = params)
    
    return qml.expval(qml.PauliZ(wires = n_qubits - 1))
#     return tuple([qml.expval(qml.PauliZ(idx)) for idx in range(n_qubits)])



In [43]:
params = np.random.randn(n_qubits - 1,requires_grad=True)
input_vec = np.random.randn(n_qubits )

result = circuit(params, input_vec)
print(circuit.draw())

 0: ──H──RY(-1.8)───╭C─────────────────────────────────────┤     
 1: ──H──RY(0.919)──│────────────╭C────────────────────────┤     
 2: ──H──RY(0.78)───│────────────│────────────╭C───────────┤     
 3: ──H──RY(2.39)───╰RY(-0.677)──╰RY(-0.367)──╰RY(-0.135)──┤ ⟨Z⟩ 



In [44]:
aparams = np.random.randn(n_qubits - 1, requires_grad=True)
input_vec = np.random.randn(n_qubits)

jac = qml.jacobian(circuit, argnum=1)(params, input_vec)

print(jac)

[ 0.19346464 -0.05283473  0.25304361  0.49014148]


In [45]:
params = np.random.randn(n_qubits - 1, requires_grad=True)
input_vec = np.random.randn(n_qubits)

grad = qml.grad(circuit, argnum=1)(params, input_vec)

print(grad)

[-0.19988774 -0.03179257 -0.08822609  0.23997154]


In [46]:
n_qubits = 2
@qml.qnode(dev)
def circuit(angle, n_qubits = n_qubits):
    qml.RY(angle, wires = 0)
    qml.CNOT(wires = [0,1])
   
    return [qml.expval(qml.PauliZ(x)) for x in range(0, n_qubits)]


In [50]:
result = circuit(np.pi/6)

print(result)
print(circuit.draw())

[0.8660254 0.8660254]
 0: ──RY(0.524)──╭C──┤ ⟨Z⟩ 
 1: ─────────────╰X──┤ ⟨Z⟩ 



In [48]:
jac = qml.jacobian(circuit, argnum = 0)(np.pi/6)

In [49]:
jac

array([-0.5, -0.5])

In [ ]:
import os 

In [ ]:
def get_directory_size(directory):
    """Returns the `directory` size in bytes."""
    total = 0
    try:
        # print("[+] Getting the size of", directory)
        for entry in os.scandir(directory):
            if entry.is_file():
                # if it's a file, use stat() function
                total += entry.stat().st_size
            elif entry.is_dir():
                # if it's a directory, recursively call this function
                total += get_directory_size(entry.path)
    except NotADirectoryError:
        # if `directory` isn't a directory, get the file size then
        return os.path.getsize(directory)
    except PermissionError:
        # if for whatever reason we can't open the folder, return 0
        return 0
    return total

def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"

In [51]:
get_size_format(get_directory_size("./models"))

NameError: name 'get_size_format' is not defined